<a href="https://colab.research.google.com/github/AdamKimhub/Msproject1/blob/colab/forfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pathlib import Path
import ast
import numpy as np
import pandas as pd
from pymatgen.core import Structure, PeriodicSite, DummySpecie
from pymatgen.analysis.local_env import MinimumDistanceNN


## Get combined file

In [ ]:
# To do this you need to handle the dataset size issue...


## Data to graphs

In [ ]:
# The combined csv with all the high and low datasets will be named: combined.csv
defects_combined_path = Path(f'dataset/combined/combined.csv')
defects_df = pd.read_csv(defects_combined_path)


In [ ]:
# Use functions in cif_to_graog to convert each row into raphs.

I think the formula that the model should get is, if you have material x missing(vacancy defect) in the lattice or replaced by material y(the substitutional defect), the energy change in that site has a specific value(formation energy). When this value is combined with other formation energies at different defect sites at distances z, you get a positive or negative change to the band gap of the host materials.
So attributes will be:
1. Defect type(vacncy or substitutional)
2. Atomic number change of the site.(from here you get the formation energy)
3. Get formation energy relation with distance. 